In [13]:
import os
output = [int(file.split(".txt")[0].strip()) for file in os.listdir("descriptiontexts4") if file!= ".DS_Store"]
output.sort()
output = [str(i) for i in output]
with open("app_ref_doc_numbers5.txt", "w") as f:
    f.write("\n".join(output))

In [35]:
import os

# Path to the folder containing the text files
folder_path = "/Users/lawrenceliu/MIT/2024Fall/6.8611/final project/embedded/restofmitalt2/batched_titleabstracts_fulltext1"

# Iterate through the files in the folder
for filename in os.listdir(folder_path):
    if filename.startswith("batch_") and filename.endswith(".npy"):
        # Extract the numeric part of the filename
        number = int(filename[len("batch_"):-len(".npy")])
        
        # Add 673 to the number
        new_number = number + 668
        
        # Construct the new filename
        new_filename = f"batch_{new_number}.npy"
        
        # Rename the file
        old_path = os.path.join(folder_path, filename)
        new_path = os.path.join(folder_path, new_filename)
        os.rename(old_path, new_path)
        print(f"Renamed: {filename} -> {new_filename}")

Renamed: batch_65.npy -> batch_733.npy
Renamed: batch_71.npy -> batch_739.npy
Renamed: batch_59.npy -> batch_727.npy
Renamed: batch_58.npy -> batch_726.npy
Renamed: batch_70.npy -> batch_738.npy
Renamed: batch_64.npy -> batch_732.npy
Renamed: batch_72.npy -> batch_740.npy
Renamed: batch_66.npy -> batch_734.npy
Renamed: batch_67.npy -> batch_735.npy
Renamed: batch_73.npy -> batch_741.npy
Renamed: batch_77.npy -> batch_745.npy
Renamed: batch_63.npy -> batch_731.npy
Renamed: batch_62.npy -> batch_730.npy
Renamed: batch_76.npy -> batch_744.npy
Renamed: batch_48.npy -> batch_716.npy
Renamed: batch_60.npy -> batch_728.npy
Renamed: batch_74.npy -> batch_742.npy
Renamed: batch_75.npy -> batch_743.npy
Renamed: batch_61.npy -> batch_729.npy
Renamed: batch_49.npy -> batch_717.npy
Renamed: batch_12.npy -> batch_680.npy
Renamed: batch_13.npy -> batch_681.npy
Renamed: batch_8.npy -> batch_676.npy
Renamed: batch_11.npy -> batch_679.npy
Renamed: batch_39.npy -> batch_707.npy
Renamed: batch_38.npy -> b

In [39]:
import numpy as np
# Reconstruct array of all title abstract embeddings
accounts = ["embedded/bookofmasters9","embedded/mitalt","embedded/mitalt2","embedded/mitalt3"]
all_titleabstracts = []
all_fulltext = []
all_fulltext_trunc = []
all_indices = []
for account in accounts:
    for a in os.listdir(os.path.join(account, "batched_indices")):
        if a == ".DS_Store":
            continue
        batch_ind = a.replace(".txt", "")
        indices_path = os.path.join(account, "batched_indices", a)
        titleabstract_path = os.path.join(account, "batched_titleabstracts", f"{batch_ind}.npy")
        full_text_path = os.path.join(account, "batched_titleabstracts_fulltext", f"{batch_ind}.npy")
        full_text_trunc_path = os.path.join(account, "batched_titleabstracts_fulltext_trunc", f"{batch_ind}.npy")
        with open(indices_path, "r") as f:
            all_indices.extend(f.read().splitlines())
        all_titleabstracts.append(np.load(titleabstract_path))
        all_fulltext.append(np.load(full_text_path))
        all_fulltext_trunc.append(np.load(full_text_trunc_path))
all_titleabstracts = np.vstack(all_titleabstracts)
all_fulltext = np.vstack(all_fulltext)
all_fulltext_trunc = np.vstack(all_fulltext_trunc)
all_indices = np.array(all_indices)
print(all_titleabstracts.shape, all_fulltext.shape, all_fulltext_trunc.shape, all_indices.shape)

(29526, 1024) (29526, 1024) (29526, 1024) (29526,)


In [40]:
# Save each of the np arrays to all folder
base_filepath = "/Users/lawrenceliu/MIT/2024Fall/6.8611/final project/embedded/all"
np.save(os.path.join(base_filepath, "titleabstracts.npy"), all_titleabstracts)
np.save(os.path.join(base_filepath, "fulltext.npy"), all_fulltext)
np.save(os.path.join(base_filepath, "fulltext_trunc.npy"), all_fulltext_trunc)
np.save(os.path.join(base_filepath, "indices.npy"), all_indices)

In [33]:
all_inds = []
for i in range(667,750):
    filename = f"batch_{i}.txt"
    with open(os.path.join("/Users/lawrenceliu/MIT/2024Fall/6.8611/final project/embedded/mitalt2/batched_indices", filename), "r") as f:
        all_inds.extend(f.read().split())
with open("temp_indices.txt","w") as g:
    g.write("\n".join(sorted(all_inds)))

In [21]:
import pandas as pd
df = pd.DataFrame(0, index=all_indices, columns=['titleabstracts', 'fulltext', 'fulltext_trunc'])

In [ ]:
accounts = ["embedded/bookofmasters9","embedded/mitalt","embedded/mitalt2","embedded/mitalt3"]
for account in accounts:
    directory_base = os.path.join(account,"batched_titleabstracts")
    for batch in os.listdir(directory_base):
        batch_filename = os.path.join(directory_base, batch)
        batch_ind_filename = os.path.join(account, "batched_indices", batch.replace(".npy", ".txt"))
        with open(batch_ind_filename, "r") as f:
            batch_indices = f.read().split()
        batch_embeddings = np.load(batch_filename)
        for ind, embedding in zip(batch_indices, batch_embeddings):
            df.at[ind, 'titleabstracts'] = embedding
# Count number of 0s in titleabstracts column of df
print(df['titleabstracts'].apply(lambda x: np.all(x==0)).sum())

In [ ]:
# Reconstruct array of all fulltext embeddings, non truncated
accounts = ["embedded/bookofmasters9","embedded/mitalt","embedded/mitalt2","embedded/restofmitalt2","embedded/mitalt3"]
all_fulltext = []
for account in accounts:
    directory_base = os.path.join(account,"batched_titleabstracts_fulltext")
    for batch in os.listdir(directory_base):
        batch_filename = os.path.join(directory_base, batch)
        index_filename = os.path.join(account, "batched_indices_fulltext",batch)
        batch_embeddings = np.load(batch_filename)
        print(batch_embeddings.shape)
        all_fulltext.append(batch_embeddings)
all_fulltext = np.vstack(all_fulltext)
print(all_fulltext.shape)